
# CC3039 — Modelación y Simulación  
## Laboratorio 7 — **Parte Práctica 2** (Modelo Híbrido: MBA + DS + DES)

- Joaquín Campos - 22155
- Sofía García - 22210
- Julio García Salas - 22076

In [1]:
import simpy
import random
import math
import numpy as np
import statistics
import json

# Semillas para reproducibilidad
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# --- Espacio y población (MBA) ---
NUM_AGENTES = 60
ANCHO_MUNDO = 100.0
ALTO_MUNDO  = 100.0
DT = 1.0                 # paso de tiempo (min)

# --- Dinámica interna (DS) ---
ENERGIA_MAX = 10.0
TASA_RECUPERACION_NATURAL = 0.05      # por minuto
GASTO_POR_MOVIMIENTO      = 0.02      # costo por (norma velocidad)*min
EFECTO_INTERACCION_BASE   = 0.02      # magnitud base ± por encuentro
RADIO_INTERACCION         = 6.0       # rango de interacción

# --- Recurso de recarga (DES) ---
ENERGIA_CRITICA = 2.0
TIEMPO_RECARGA  = 10.0

# --- Horizonte de simulación ---
TIEMPO_SIMULACION = 240.0   # minutos totales


In [2]:

# ============================
# 2) Clase Agente (MBA + DS + DES)
# ============================
class Agente:
    def __init__(self, idx, env, estacion_recarga):
        self.id = idx
        self.env = env
        self.estacion = estacion_recarga

        # Estado MBA
        self.pos = np.array([random.uniform(0, ANCHO_MUNDO), random.uniform(0, ALTO_MUNDO)], dtype=float)
        ang = random.uniform(0, 2*math.pi)
        speed = 1.0 + random.uniform(-0.5, 0.5)   # módulo velocidad ~ 0.5–1.5
        self.vel = np.array([math.cos(ang), math.sin(ang)], dtype=float) * speed

        # Estado DS
        self.energia = ENERGIA_MAX

        # Estado DES / control
        self.en_cola_o_cargando = False
        self.sesiones_recarga = 0

        # Métricas individuales
        self.tiempo_bajo_critico = 0.0

    def proceso_recarga(self, registro_esperas):
        """ Flujo DES: esperar puesto -> recargar -> salir. """
        self.en_cola_o_cargando = True
        t0 = self.env.now
        with self.estacion.request() as req:
            yield req
            espera = self.env.now - t0
            registro_esperas.append({"id": self.id, "espera": espera})
            # Servicio de recarga
            yield self.env.timeout(TIEMPO_RECARGA)
            self.energia = ENERGIA_MAX
            self.sesiones_recarga += 1
        self.en_cola_o_cargando = False

    def actualizar(self, efecto_interaccion):
        """ Un paso de actualización híbrido (MBA+DS) para este agente. """
        # DS: integrar energía
        gasto_mov = GASTO_POR_MOVIMIENTO * np.linalg.norm(self.vel)
        delta_e = (TASA_RECUPERACION_NATURAL - gasto_mov + efecto_interaccion) * DT
        self.energia = max(0.0, min(ENERGIA_MAX, self.energia + delta_e))

        # MBA: mover y rebotar en bordes
        self.pos += self.vel * DT
        if self.pos[0] < 0 or self.pos[0] > ANCHO_MUNDO:
            self.vel[0] *= -1
            self.pos[0] = max(0, min(ANCHO_MUNDO, self.pos[0]))
        if self.pos[1] < 0 or self.pos[1] > ALTO_MUNDO:
            self.vel[1] *= -1
            self.pos[1] = max(0, min(ALTO_MUNDO, self.pos[1]))


In [3]:

# ============================
# 3) Simulación principal (ejecutor híbrido)
# ============================
def ejecutar_simulacion_hibrida(num_puestos_recarga=5, tiempo_sim=TIEMPO_SIMULACION, semilla=RANDOM_SEED):
    random.seed(semilla)
    np.random.seed(semilla)

    env = simpy.Environment()
    estacion = simpy.Resource(env, capacity=num_puestos_recarga)

    # Agentes
    agentes = [Agente(i, env, estacion) for i in range(NUM_AGENTES)]

    # Series temporales 
    serie_energy_avg = []     # energía promedio de la población
    serie_frac_bajo  = []     # fracción de agentes con energía < crítica
    serie_q_len      = []     # longitud de cola en la estación
    serie_en_serv    = []     # concurrentes en servicio (uso de estación)

    # Sesiones de recarga
    registro_esperas = []     # dicts {"id":..., "espera":...}

    def bucle_principal():
        while env.now < tiempo_sim:
            # 1) Interacciones: efecto por agente (simétrica, ± con decaimiento por distancia)
            efectos = np.zeros(NUM_AGENTES, dtype=float)
            for i in range(NUM_AGENTES):
                pi = agentes[i].pos
                for j in range(i+1, NUM_AGENTES):
                    pj = agentes[j].pos
                    d = np.linalg.norm(pi - pj)
                    if d <= RADIO_INTERACCION:
                        e = (random.choice([-1, 1])) * EFECTO_INTERACCION_BASE * (1 - d/RADIO_INTERACCION)
                        efectos[i] += e
                        efectos[j] += e

            # 2) Actualizar y disparar DES si crítico
            bajos = 0
            for k, ag in enumerate(agentes):
                ag.actualizar(efectos[k])
                if ag.energia < ENERGIA_CRITICA:
                    bajos += 1
                    ag.tiempo_bajo_critico += DT
                    if not ag.en_cola_o_cargando:
                        env.process(ag.proceso_recarga(registro_esperas))

            # 3) Muestreos del recurso
            serie_q_len.append(len(estacion.queue))
            serie_en_serv.append(estacion.count)

            # 4) Muestreos de energía
            energia_prom = sum(a.energia for a in agentes) / NUM_AGENTES
            serie_energy_avg.append(energia_prom)
            serie_frac_bajo.append(bajos / NUM_AGENTES)

            # 5) Avance de tiempo
            yield env.timeout(DT)

    env.process(bucle_principal())
    env.run(until=tiempo_sim)

    # Esperas (por sesión)
    esperas = [r["espera"] for r in registro_esperas]
    prom_espera = statistics.mean(esperas) if esperas else 0.0
    p50_espera  = statistics.median(esperas) if esperas else 0.0
    p90_espera  = float(np.percentile(esperas, 90)) if esperas else 0.0
    max_espera  = max(esperas) if esperas else 0.0

    # Uso y cola
    prom_en_serv = statistics.mean(serie_en_serv) if serie_en_serv else 0.0
    prom_cola    = statistics.mean(serie_q_len) if serie_q_len else 0.0
    max_cola     = max(serie_q_len) if serie_q_len else 0

    # Energía
    prom_energia_t = statistics.mean(serie_energy_avg) if serie_energy_avg else 0.0
    min_energia_t  = min(serie_energy_avg) if serie_energy_avg else 0.0
    frac_bajo_prom = statistics.mean(serie_frac_bajo) if serie_frac_bajo else 0.0
    frac_bajo_max  = max(serie_frac_bajo) if serie_frac_bajo else 0.0

    agentes_unicos = len({r["id"] for r in registro_esperas})

    resultados = {
        "parametros": {
            "NUM_AGENTES": NUM_AGENTES,
            "NUM_PUESTOS_RECARGA": num_puestos_recarga,
            "ENERGIA_CRITICA": ENERGIA_CRITICA,
            "TIEMPO_RECARGA": TIEMPO_RECARGA,
            "DT": DT,
            "TIEMPO_SIMULACION": tiempo_sim,
        },
        "uso_estacion": {
            "total_sesiones": len(esperas),
            "agentes_unicos": agentes_unicos,
            "promedio_concurrente_en_servicio": prom_en_serv,
            "promedio_largo_cola": prom_cola,
            "max_largo_cola": max_cola,
        },
        "esperas": {
            "promedio_espera": prom_espera,
            "p50_espera": p50_espera,
            "p90_espera": p90_espera,
            "max_espera": max_espera
        },
        "energia": {
            "promedio_energia_poblacion": prom_energia_t,
            "min_energia_promedio": min_energia_t,
            "fraccion_promedio_bajo_critico": frac_bajo_prom,
            "fraccion_max_bajo_critico": frac_bajo_max
        }
    }
    return resultados


In [4]:

# ============================
# 4) Escenarios A y B, impresión de métricas
# ============================
def pretty_print(titulo, res):
    print(f"\n=== {titulo} ===")
    print(f"- Sesiones de recarga: {res['uso_estacion']['total_sesiones']} (agentes únicos: {res['uso_estacion']['agentes_unicos']})")
    print(f"- Uso promedio estación (concurrentes): {res['uso_estacion']['promedio_concurrente_en_servicio']:.3f}")
    print(f"- Cola promedio: {res['uso_estacion']['promedio_largo_cola']:.3f}  | Cola máx.: {res['uso_estacion']['max_largo_cola']}")
    print(f"- Espera promedio: {res['esperas']['promedio_espera']:.3f} min | P50: {res['esperas']['p50_espera']:.3f} | P90: {res['esperas']['p90_espera']:.3f} | Máx: {res['esperas']['max_espera']:.3f}")
    print(f"- Energía promedio de la población: {res['energia']['promedio_energia_poblacion']:.3f}")
    print(f"- Fracción prom. bajo crítico: {res['energia']['fraccion_promedio_bajo_critico']:.3f} | Fracción máx. bajo crítico: {res['energia']['fraccion_max_bajo_critico']:.3f}")

# Escenario A: NUM_PUESTOS_RECARGA = 5
res_cap5 = ejecutar_simulacion_hibrida(num_puestos_recarga=5, tiempo_sim=TIEMPO_SIMULACION)

# Escenario B: NUM_PUESTOS_RECARGA = 1
res_cap1 = ejecutar_simulacion_hibrida(num_puestos_recarga=1, tiempo_sim=TIEMPO_SIMULACION)

pretty_print("Escenario A — NUM_PUESTOS_RECARGA = 5", res_cap5)
pretty_print("Escenario B — NUM_PUESTOS_RECARGA = 1", res_cap1)

comparativo = {
    "cap5": res_cap5,
    "cap1": res_cap1,
    "delta": {
        "energia_promedio_poblacion": res_cap5["energia"]["promedio_energia_poblacion"] - res_cap1["energia"]["promedio_energia_poblacion"],
        "espera_promedio": res_cap5["esperas"]["promedio_espera"] - res_cap1["esperas"]["promedio_espera"],
        "cola_promedio": res_cap5["uso_estacion"]["promedio_largo_cola"] - res_cap1["uso_estacion"]["promedio_largo_cola"],
        "concurrentes_promedio": res_cap5["uso_estacion"]["promedio_concurrente_en_servicio"] - res_cap1["uso_estacion"]["promedio_concurrente_en_servicio"],
        "fraccion_promedio_bajo_critico": res_cap5["energia"]["fraccion_promedio_bajo_critico"] - res_cap1["energia"]["fraccion_promedio_bajo_critico"],
    }
}

print("\n=== Bloque de resultados ===")
print(json.dumps(comparativo, indent=2, ensure_ascii=False))



=== Escenario A — NUM_PUESTOS_RECARGA = 5 ===
- Sesiones de recarga: 0 (agentes únicos: 0)
- Uso promedio estación (concurrentes): 0.000
- Cola promedio: 0.000  | Cola máx.: 0
- Espera promedio: 0.000 min | P50: 0.000 | P90: 0.000 | Máx: 0.000
- Energía promedio de la población: 10.000
- Fracción prom. bajo crítico: 0.000 | Fracción máx. bajo crítico: 0.000

=== Escenario B — NUM_PUESTOS_RECARGA = 1 ===
- Sesiones de recarga: 0 (agentes únicos: 0)
- Uso promedio estación (concurrentes): 0.000
- Cola promedio: 0.000  | Cola máx.: 0
- Espera promedio: 0.000 min | P50: 0.000 | P90: 0.000 | Máx: 0.000
- Energía promedio de la población: 10.000
- Fracción prom. bajo crítico: 0.000 | Fracción máx. bajo crítico: 0.000

=== Bloque de resultados ===
{
  "cap5": {
    "parametros": {
      "NUM_AGENTES": 60,
      "NUM_PUESTOS_RECARGA": 5,
      "ENERGIA_CRITICA": 2.0,
      "TIEMPO_RECARGA": 10.0,
      "DT": 1.0,
      "TIEMPO_SIMULACION": 240.0
    },
    "uso_estacion": {
      "total_sesio

**Parámetros base:** NUM_AGENTES=60, DT=1.0 min, TIEMPO_SIMULACION=240.0 min, ENERGIA_CRITICA=2.0, TIEMPO_RECARGA=10.0 min.

### Resumen de métricas observadas
| Escenario | Sesiones de recarga | Cola prom. | Uso prom. (concurrentes) | Espera prom. (min) | Energía prom. población | Fracción prom. < crítica |
|---|---:|---:|---:|---:|---:|---:|
| Capacidad 5 | 0 | 0 | 0.000 | 0.000 | 10.000 | 0.000 |
| Capacidad 1 | 0 | 0 | 0.000 | 0.000 | 10.000 | 0.000 |

---

### Q1) Escenario con NUM_PUESTOS_RECARGA = 5
- **Uso observado:** 0 sesiones; 0 agentes únicos. **Colas:** inexistentes (promedio 0, máx. 0).  
- **Interpretación:** el sistema opera en régimen donde los agentes no caen bajo el umbral energético; la estación permanece ociosa.

### Q2) Escenario con NUM_PUESTOS_RECARGA = 1 (capacidad drásticamente reducida)
- **Impacto en dinámica energética:** **nulo** en estas corridas (energía promedio ≈ 10.000; fracción bajo crítica ≈ 0).  
- **Explicación:** con los parámetros actuales, el balance energético **neto es positivo** (recuperación natural > gasto por movimiento en promedio; interacciones con media ~0), por lo que casi ningún agente cruza el umbral crítico y no se activa la cola/despacho DES.

### Q3) Efecto del cuello de botella (cuando NUM_PUESTOS_RECARGA = 1)
- **No se observa cuello de botella**: esperas, colas y uso promedio se mantienen en 0; la energía agregada no se degrada.  
- **Nota metodológica:** para inducir cuello de botella y observar efectos agregados (energía promedio decreciente, mayor fracción bajo crítica) se sugiere explorar un régimen más exigente:  
  - aumentar **GASTO_POR_MOVIMIENTO** (p.ej., 0.04–0.06),  
  - reducir **TASA_RECUPERACION_NATURAL** (p.ej., 0.01–0.02),  
  - sesgar **EFECTO_INTERACCION_BASE** a negativo o incrementar **RADIO_INTERACCION**,  
  - aumentar el horizonte (**TIEMPO_SIMULACION**) para permitir acumulación.

### Q4) Bucle de retroalimentación DES ↔ MBA+DS (interpretación conceptual)
- **Camino DES→MBA+DS**: menor capacidad de recarga → **colas y esperas** mayores → más tiempo con energía baja → **menor movilidad/alcance** y más encuentros desfavorables → caída del promedio de energía.  
- **Camino MBA+DS→DES**: más agentes con energía baja → **más llegadas** a la estación → mayor utilización y **posible saturación** del recurso.  
- **Síntesis**: Aunque en estas ejecuciones el bucle no se activó, el marco híbrido ilustra que **la dinámica conjunta** puede generar pérdida de desempeño sistémico incluso si cada componente aislado parecería estable (“el todo es más que la suma de sus partes”).

---

**Observación final:** para reportes robustos, conviene ejecutar **réplicas** y resumir con **promedios y percentiles (P50/P90)** de: uso de estación, espera y fracción bajo crítica.
